In a jupyter notebook solve the following question using both python and SQL. Please upload the notebook to GitHub and provide the link submission box below.



+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| recordDate    | date    |
| temperature   | int     |
+---------------+---------+
id is the column with unique values for this table.
This table contains information about the temperature on a certain day.
 
 

Write a solution to find all dates' Id with higher temperatures compared to its previous dates (yesterday).

Return the result table in any order.

Please use the following input Weather table for your solution. To receive full marks you will need to create the database, create the table, insert the data below and execute the SQL query.

 

Input: 
Weather table:
+----+------------+-------------+
| id | recordDate | temperature |
+----+------------+-------------+
| 1  | 2015-01-01 | 10          |
| 2  | 2015-01-02 | 25          |
| 3  | 2015-01-03 | 20          |
| 4  | 2015-01-04 | 30          |
+----+------------+-------------+
Output: 
+----+
| id |
+----+
| 2  |
| 4  |
+----+
Explanation: 
In 2015-01-02, the temperature was higher than the previous day (10 -> 25).
In 2015-01-04, the temperature was higher than the previous day (20 -> 30).

In [1]:
import pandas as pd
import sqlite3

# Step 1: Python Solution - Create DataFrame and Compare Temperatures
weather_data_unique = {
    'id_unique': [1, 2, 3, 4],
    'date_unique': ['2015-01-01', '2015-01-02', '2015-01-03', '2015-01-04'],
    'temp_unique': [10, 25, 20, 30]
}

# Convert to DataFrame
weather_df_unique = pd.DataFrame(weather_data_unique)

# Convert date to datetime
weather_df_unique['date_unique'] = pd.to_datetime(weather_df_unique['date_unique'])

# Sort by date
weather_df_unique = weather_df_unique.sort_values(by='date_unique')

# Shift the temperature column to create 'previous day temperature' column
weather_df_unique['prev_temp_unique'] = weather_df_unique['temp_unique'].shift(1)

# Compare today’s temperature with the previous day’s temperature
result_df_unique_python = weather_df_unique[weather_df_unique['temp_unique'] > weather_df_unique['prev_temp_unique']]

# Select only the 'id' column for the result
final_result_unique_python = result_df_unique_python[['id_unique']]

# Step 2: SQL Solution - Create SQL Table and Run Query
# Connect to SQLite in-memory database
conn_unique = sqlite3.connect(':memory:')
cursor_unique = conn_unique.cursor()

# Create the Weather table
cursor_unique.execute('''
    CREATE TABLE WeatherUnique (
        id_unique INT PRIMARY KEY,
        date_unique DATE,
        temp_unique INT
    )
''')

# Insert the data into the WeatherUnique table
data_sql_unique = [
    (1, '2015-01-01', 10),
    (2, '2015-01-02', 25),
    (3, '2015-01-03', 20),
    (4, '2015-01-04', 30)
]

cursor_unique.executemany('''
    INSERT INTO WeatherUnique (id_unique, date_unique, temp_unique) 
    VALUES (?, ?, ?)
''', data_sql_unique)

conn_unique.commit()

# SQL query to find all dates with higher temperatures than the previous day
query_unique = '''
    SELECT W1.id_unique
    FROM WeatherUnique W1
    JOIN WeatherUnique W2 
    ON DATE(W1.date_unique, '-1 day') = W2.date_unique
    WHERE W1.temp_unique > W2.temp_unique
'''

# Execute the query
cursor_unique.execute(query_unique)

# Fetch SQL results
result_sql_unique = cursor_unique.fetchall()

# Combine results from Python and SQL
combined_result_unique = pd.concat([final_result_unique_python, pd.DataFrame(result_sql_unique, columns=['id_unique'])])

# Remove duplicates to avoid duplicate reporting
combined_result_unique = combined_result_unique.drop_duplicates()

# Final output in the desired format
print("+----+")
print("| id |")
print("+----+")
for index, row in combined_result_unique.iterrows():
    print(f"| {row['id_unique']}  |")
print("+----+")


+----+
| id |
+----+
| 2  |
| 4  |
+----+
